In [19]:
import numpy as np
from scipy import stats
import pandas as pd
import numpy as np

In [3]:
ghg_df = pd.read_csv('/home/599/dbi599/ocean-analysis/visualisation/water_cycle/GHG-only.csv')

In [4]:
ghg_df

,Unnamed: 0,model,component,P-E region,"time integrated P-E anomaly, 1861-2005 (kg)"
0,0,CanESM2,total,SH-P,1.253944e+17
1,1,CanESM2,total,SH-E,-1.952247e+17
2,2,CanESM2,total,T-P,4.443918e+16
3,3,CanESM2,total,NH-E,-9.373015e+16
4,4,CanESM2,total,NH-P,1.175894e+17
...,...,...,...,...,...
265,265,GFDL-ESM4,area,SH-P,-2.900105e+16
266,266,GFDL-ESM4,area,SH-E,-7.284136e+16
267,267,GFDL-ESM4,area,T-P,1.024520e+16
268,268,GFDL-ESM4,area,NH-E,-3.132190e+15


In [11]:
shp_total = ghg_df[(ghg_df['P-E region'] == 'SH-P') & (ghg_df['component'] == 'total')]['time integrated P-E anomaly, 1861-2005 (kg)'].values

In [17]:
def mean_confidence_interval(data, confidence=0.95):
    """Calculate 95% CI on the mean.

    The interval is the mean +/- h.

    """

    n = len(data)
    se = stats.sem(data)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)

    return h

In [18]:
mean_confidence_interval(shp_total)

2.589153811874242e+16

In [20]:
lower = np.array([9.011262033699773e+16, -3.321220393544478e+17, 1.2242470655152462e+17, -1.859302540309282e+17, 8.462133035645846e+16, 1.1213825433056557e+17, -2.4441290521975392e+17, 1.2177713258388851e+17, -1.6360380662228675e+17, 9.913872721075598e+16, -3.0620867720081524e+16, -1.4097081522833664e+17, -6.377824095347973e+16, -7.019173487347237e+16, -1.7358645248886456e+16])
upper = np.array([1.3810471948461789e+17, -2.079246228138969e+17, 2.487269082170289e+17, -8.719099157509086e+16, 1.2974817663673728e+17, 1.6707882110843424e+17, -1.4043007048788402e+17, 2.808198307819691e+17, -7.175461673613359e+16, 1.4215261108641176e+17, 1076643060136690.1, -6.257978055492699e+16, 6.421792834125952e+16, -7901254724873188.0, 1.270767930647456e+16])


In [22]:
upper - lower

array([4.79920991e+16, 1.24197417e+17, 1.26302202e+17, 9.87392625e+16,
       4.51268463e+16, 5.49405668e+16, 1.03982835e+17, 1.59042698e+17,
       9.18491899e+16, 4.30138839e+16, 3.16975108e+16, 7.83910347e+16,
       1.27996169e+17, 6.22904801e+16, 3.00663246e+16])

In [ ]:
def bootstrap(*args, **kwargs):
    """Resample one or more arrays with replacement and store aggregate values.
    Positional arguments are a sequence of arrays to bootstrap along the first
    axis and pass to a summary function.
    Keyword arguments:
        n_boot : int, default=10000
            Number of iterations
        axis : int, default=None
            Will pass axis to ``func`` as a keyword argument.
        units : array, default=None
            Array of sampling unit IDs. When used the bootstrap resamples units
            and then observations within units instead of individual
            datapoints.
        func : string or callable, default="mean"
            Function to call on the args that are passed in. If string, uses as
            name of function in the numpy namespace. If nans are present in the
            data, will try to use nan-aware version of named function.
        seed : Generator | SeedSequence | RandomState | int | None
            Seed for the random number generator; useful if you want
            reproducible resamples.
    Returns
    -------
    boot_dist: array
        array of bootstrapped statistic values
    """
    # Ensure list of arrays are same length
    if len(np.unique(list(map(len, args)))) > 1:
        raise ValueError("All input arrays must have the same length")
    n = len(args[0])

    # Default keyword arguments
    n_boot = kwargs.get("n_boot", 10000)
    func = kwargs.get("func", "mean")
    axis = kwargs.get("axis", None)
    units = kwargs.get("units", None)
    random_seed = kwargs.get("random_seed", None)
    if random_seed is not None:
        msg = "`random_seed` has been renamed to `seed` and will be removed"
        warnings.warn(msg)
    seed = kwargs.get("seed", random_seed)
    if axis is None:
        func_kwargs = dict()
    else:
        func_kwargs = dict(axis=axis)

    # Initialize the resampler
    rng = _handle_random_seed(seed)

    # Coerce to arrays
    args = list(map(np.asarray, args))
    if units is not None:
        units = np.asarray(units)

    if isinstance(func, str):

        # Allow named numpy functions
        f = getattr(np, func)

        # Try to use nan-aware version of function if necessary
        missing_data = np.isnan(np.sum(np.column_stack(args)))

        if missing_data and not func.startswith("nan"):
            nanf = getattr(np, f"nan{func}", None)
            if nanf is None:
                msg = f"Data contain nans but no nan-aware version of `{func}` found"
                warnings.warn(msg, UserWarning)
            else:
                f = nanf

    else:
        f = func

    # Handle numpy changes
    try:
        integers = rng.integers
    except AttributeError:
        integers = rng.randint

    # Do the bootstrap
    if units is not None:
        return _structured_bootstrap(args, n_boot, units, f,
                                     func_kwargs, integers)

    boot_dist = []
    for i in range(int(n_boot)):
        resampler = integers(0, n, n, dtype=np.intp)  # intp is indexing dtype
        sample = [a.take(resampler, axis=0) for a in args]
        boot_dist.append(f(*sample, **func_kwargs))
    return np.array(boot_dist)


In [ ]:
boots = bootstrap(stat_data, func=estimator,
                  n_boot=n_boot,
                  units=unit_data)
utils.ci(boots, ci)

In [26]:
import seaborn

In [28]:
seaborn.algorithms.bootstrap

<function seaborn.algorithms.bootstrap(*args, **kwargs)>

In [42]:
boots = seaborn.algorithms.bootstrap(shp_total)
cis = seaborn.utils.ci(boots, 95)
#seaborn.utils.ci_to_errsize(cis, np.mean(shp_total))

In [43]:
cis

array([9.07422338e+16, 1.37617071e+17])

In [39]:
np.mean(shp_total)

1.1227081159113846e+17

In [46]:
diff = cis[1] - cis[0]

In [49]:
np.mean(shp_total) + (diff/2)

1.3570823004204166e+17

In [50]:
np.mean(shp_total) - (diff/2)

8.883339314023526e+16

In [51]:
diff

4.687483690180642e+16